## <u>Cuckoo Filter</u>
### Maggie Drew & Adam Gibbs

Implementation of Cuckoo Filter for estimating set membership on data streams. Implementation psuedocode from the following paper: https://www.cs.cmu.edu/~dga/papers/cuckoo-conext2014.pdf. This notebook contains the following:

1. Imports used in the notebook
2. All global variables and python data structures (lists, dictionaries, etc.) used
3. Code for Cuckoo Filter functions
4. Cuckoo filter test on simulated data stream
5. Results
6. Analysis of Results 

In [7]:
# IMPORTS
import matplotlib as plt
import numpy as np 
import gzip


In [ ]:
# VARIABLES and DATA STRUCTURES
dataset = './datasets/accidents.dat.gz'

In [ ]:
# IMPLEMENTATION OF CUCKOO FILTER METHODS

In [ ]:
# READ IN DATA AND APPLY FILTER

with gzip.open(dataset, 'rt') as data_stream:
    for line in data_stream:
        for element in line.split():
            # apply cuckoo filter

In [ ]:
# ANALYSIS OF RESULTS